In [16]:
import pandas as pd
import numpy as np
import cv2
import os
import wisardpkg as wp
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [36]:
#descompactar o zip
if len(os.listdir('./dataset')) <= 2: 
    !unzip -qq ./dataset/HAR.zip -d ./dataset/HAR

In [28]:
def load_images(filenames, labels, folder):
    images = []
    for filename in filenames:
        img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (28, 28))  # Reescalona a imagem para 28 x 28. Experimentem com outros tamanhos
            images.append(img.flatten())  # Achata a imagem
    return images

In [38]:
root_path = './dataset/HAR/Human_Action_Recognition/'
train_df = pd.read_csv(root_path + 'Training_set.csv')

# Filtros para classes de interesse
classes_of_interest = ['hugging', 'running', 'texting', 'fighting']
filtered_df = train_df[train_df['label'].isin(classes_of_interest)]

train_folder = root_path + 'train'

# Carrega as imagens do dataset filtrado
train_images = load_images(filtered_df['filename'].tolist(), filtered_df['label'].tolist(), train_folder)


In [39]:
# Binarização
train_images_binary = [list(map(int, np.round(img / 255))) for img in train_images]

# Rótulos
train_labels = filtered_df['label'].tolist()

In [40]:
X_train, X_test, y_train, y_test = train_test_split(train_images_binary, train_labels, test_size=0.3, random_state=42)

In [41]:
num_bits_addr = 14
model = wp.Wisard(num_bits_addr)

In [42]:
model.train(X_train, y_train)

In [43]:
y_pred = model.classify(X_test)

acc = accuracy_score(y_test, y_pred)
print('acc: ', acc)

acc:  0.44345238095238093
